# Transfroming the Dicom Scans inton PNGs and collecting them in a folder

In [1]:
import os
import shutil
import pydicom
from PIL import Image
from pydicom.errors import InvalidDicomError

def extract_number(filename):
    number_part = ''.join(filter(str.isdigit, filename))
    return int(number_part) if number_part else 0

def extract_dicom_parameters(dicom_path):
    try:
        dicom_file = pydicom.dcmread(dicom_path)
        pixel_spacing = dicom_file.PixelSpacing if "PixelSpacing" in dicom_file else "Not available"
        slice_thickness = dicom_file.SliceThickness if "SliceThickness" in dicom_file else "Not available"
        return pixel_spacing, slice_thickness
    except Exception as e:
        return str(e), None

def dicom_folder_to_png(dicom_folder, print_progress=True, print_parameters=True):
    output_folder = "PNGS_Final"

    # Check if the output folder exists and delete it
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)

    # Create a new empty output folder
    os.makedirs(output_folder, exist_ok=True)

    dicom_files = sorted([f for f in os.listdir(dicom_folder)], key=extract_number)
    dicom_parameters = None

    for idx, dicom_file in enumerate(dicom_files):
        dicom_path = os.path.join(dicom_folder, dicom_file)

        if print_progress:
            print(f"Converting {dicom_path}...")

        try:
            with pydicom.dcmread(dicom_path) as ds:
                # Extract DICOM parameters only once
                if dicom_parameters is None and print_parameters:
                    dicom_parameters = extract_dicom_parameters(dicom_path)
                    print(f"DICOM Parameters (Pixel Spacing, Slice Thickness): {dicom_parameters}")

                pixel_data = ds.pixel_array
                normalized_pixel_data = ((pixel_data - pixel_data.min()) / (pixel_data.max() - pixel_data.min()) * 255).astype('uint8')
                img = Image.fromarray(normalized_pixel_data, mode='L')
                dpi = 512
                png_file = os.path.join(output_folder, f"{idx + 1:03d}.png")
                img.save(png_file, "PNG", dpi=(dpi, dpi))
                
                if print_progress:
                    print(f"Saved {png_file}")

        except InvalidDicomError:
            if print_progress:
                print(f"Skipped non-DICOM file: {dicom_file}")

dicom_folder = "AXIAL"
dicom_folder_to_png(dicom_folder, print_progress=False, print_parameters=True)


DICOM Parameters (Pixel Spacing, Slice Thickness): ([0.415, 0.415], '0.833')


# Classifying the Trackers into Clusters

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

def find_circles(image_path, display=False):
    # Read image
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # Adjust the brightness of the image
    brightness_factor = 2
    enhanced_img = cv2.convertScaleAbs(img, alpha=brightness_factor, beta=0)

    # Convert the enhanced image to grayscale
    gray = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2GRAY)

    # Binarize the grayscale image
    _, binary_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # Fill in the shapes (contours) in the binary image
    binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize a list to store valid circles
    valid_circles = []

    for contour in contours:
        # Calculate the area and perimeter for each contour
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)

        # Skip contour if the perimeter is zero (to avoid division by zero)
        if perimeter == 0:
            continue

        # Calculate the circularity for each contour
        circularity = 4 * np.pi * area / (perimeter * perimeter)

        # Define circularity and area thresholds to filter circles
        circularity_threshold = 0.8  # Adjust this threshold as needed
        min_area = 100  # Adjust this threshold as needed
        max_area = 2000

        # Check if the contour meets the circularity and area criteria
        if circularity >= circularity_threshold and max_area >= area >= min_area:
            # Fit a circle to the contour and obtain its center (a, b) and radius (r)
            (a, b), r = cv2.minEnclosingCircle(contour)
            a, b, r = int(a), int(b), int(r)
            valid_circles.append((a, b, r))
            
            if display:
                print(f"Circle: Center ({a}, {b}), Slice # {r}, Circularity {circularity}")
                # Draw the circumference of the circle on the image
                cv2.circle(img, (a, b), r, (255, 100, 255), 2)
                # Draw a small circle at the center
                cv2.circle(img, (a, b), 1, (255, 0, 0), 2)

    # Display the image with circles drawn if display is True
    if valid_circles and display:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert to RGB for displaying
        plt.show()

    return valid_circles


def cluster_trackers(png_files, threshold=10, display=False, show_images=False):
    all_circles = []

    for idx, png_file in enumerate(png_files):
        png_path = os.path.join(pngs_folder, png_file)
        circles = find_circles(png_path, display=display)

        if circles is not None:
            for circle in circles:
                all_circles.append((circle[0], circle[1], idx))  # x, y, z

    clusters = {}
    for circle in all_circles:
        added_to_cluster = False
        for cluster_id, cluster_circles in clusters.items():
            if all(np.linalg.norm(np.array(circle[:2]) - np.array(c[:2])) < threshold for c in cluster_circles):
                cluster_circles.append(circle)
                added_to_cluster = True
                break

        if not added_to_cluster:
            clusters[len(clusters)] = [circle]

    final_clusters = {}
    for cluster_id, cluster_circles in clusters.items():
        if len(cluster_circles) >= 6:
            # Sort the cluster circles based on the slice number (z-coordinate)
            sorted_circles = sorted(cluster_circles, key=lambda x: x[2])

            # Find the two median slices
            mid_index = len(sorted_circles) // 2
            median_circles = sorted_circles[mid_index - 1: mid_index + 1]

            # Compute the average coordinates of the two median circles
            avg_x = np.mean([c[0] for c in median_circles])
            avg_y = np.mean([c[1] for c in median_circles])
            avg_z = np.mean([c[2] for c in median_circles])
            final_clusters[cluster_id] = (avg_x, avg_y, avg_z)

            # Rest of the code remains the same...

    return final_clusters


# Folder containing PNG files
pngs_folder = 'PNGS_Final'

# List PNG files in the folder
png_files = [f for f in os.listdir(pngs_folder) if f.lower().endswith('.png')]

# Cluster the trackers with the option to show images
tracker_clusters = cluster_trackers(png_files, display=False, show_images=False)

for cluster_id, coordinates in tracker_clusters.items():
    print(f"Cluster {cluster_id}: Average Coordinates: {coordinates}")

TypeError: 'numpy._DTypeMeta' object is not subscriptable

# Defining the Tracker Object, name, shape, ...

In [27]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class Tracker:
    def __init__(self, name, tracker_type, ball_locations=None):
        self.name = name
        self.tracker_type = tracker_type
        self.ball_locations = ball_locations if ball_locations is not None else [(0, 0, 0)] * 4

    def __str__(self):
        return f"Tracker Name: {self.name}, Type: {self.tracker_type}, Ball Locations: {self.ball_locations}"

# Define the known tracker shapes with new names
tracker_shapes = {
    "cross_shape_1": [
        (0.00, 0.00, 0.00),
        (0.00, 28.59, 41.02),
        (0.00, 0.00, 88.00),
        (0.00, -44.32, 40.45)
    ],
    "cross_shape_2": [
        (0.00, 0.00, 0.00),
        (0.00, 47.38, 28.99),
        (0.00, 0.00, 89.10),
        (0.00, -35.36, 35.36)
    ]
}

# List to store the tracker objects
trackers = []

# Calculate the number of trackers
num_balls = len(tracker_balls)
num_trackers = num_balls // 4

# Display tracker shapes and ask the user for the type of each tracker
for i in range(1, num_trackers + 1):
    print("Tracker Shapes:")
    for j, shape in enumerate(tracker_shapes, start=1):
        print(f"{j}: {shape}")

    tracker_type_index = int(input(f"Select the type for Tracker {i} (Enter the number): ")) - 1
    tracker_type = list(tracker_shapes.keys())[tracker_type_index]

    # Create a Tracker object and add it to the list
    tracker = Tracker(f"Tracker_{i}", tracker_type)
    trackers.append(tracker)

# Display the details of each tracker
for tracker in trackers:
    print(tracker)




Tracker Shapes:
1: cross_shape_1
2: cross_shape_2
Select the type for Tracker 1 (Enter the number): 1
Tracker Shapes:
1: cross_shape_1
2: cross_shape_2
Select the type for Tracker 2 (Enter the number): 2
Tracker Name: Tracker_1, Type: cross_shape_1, Ball Locations: [(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)]
Tracker Name: Tracker_2, Type: cross_shape_2, Ball Locations: [(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)]


In [21]:
%matplotlib notebook

import numpy as np
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the known tracker shapes
tracker_shapes = {
    "1": [
        (0.00, 0.00, 0.00),
        (0.00, 28.59, 41.02),
        (0.00, 0.00, 88.00),
        (0.00, -44.32, 40.45)
    ],
    "2": [
        (0.00, 0.00, 0.00),
        (0.00, 47.38, 28.99),
        (0.00, 0.00, 89.10),
        (0.00, -35.36, 35.36)
    ]
}

def distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def classify_tracker_balls(tracker_balls, shape, tracker_id):
    """
    Classify each ball of the tracker based on the chosen shape.
    """
    classified_trackers = []

    # Iterate over all combinations of 4 balls
    for combination in itertools.combinations(tracker_balls, 4):
        distances = [distance(combination[i], shape[j]) for i in range(4) for j in range(4)]
        if all(d < threshold for d in distances):
            classified_trackers.append({
                "balls": [{"id": f"Tracker_{tracker_id}_ball_{i+1}", "coordinates": ball} for i, ball in enumerate(combination)],
                "shape": shape
            })

    return classified_trackers

# Extract the coordinates
tracker_balls = [coordinates for _, coordinates in tracker_clusters.items()]

# Determine the number of trackers
num_trackers = len(tracker_balls) // 4

classified_trackers = []
for i in range(1, num_trackers + 1):
    # User input for each tracker shape
    user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
    chosen_shape = tracker_shapes[user_choice]

    # Classify the tracker balls
    classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
    classified_trackers.extend(classified_tracker)

# Print the details of each tracker
for tracker in classified_trackers:
    print(f"Tracker {tracker['balls'][0]['id'].split('_')[1]}:")
    for ball in tracker['balls']:
        print(f"{ball['id']}: {ball['coordinates']}")
    print(f"Shape: {tracker['shape']}")
    
def calculate_centroid(points):
    """Calculate the centroid of a set of points."""
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    z_coords = [p[2] for p in points]
    centroid = (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    return centroid

def plot_trackers(trackers):
    """
    Plot multiple trackers with their balls in 3D.
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for tracker in trackers:
        # Extract coordinates for plotting
        x_coords = [ball[0] for ball in tracker['balls']]
        y_coords = [ball[1] for ball in tracker['balls']]
        z_coords = [ball[2] for ball in tracker['balls']]

        # Plot the balls
        ax.scatter(x_coords, y_coords, z_coords, color='b', s=50)

        # Calculate and plot centroid
        centroid = calculate_centroid(tracker['balls'])
        ax.scatter(*centroid, color='r', s=100)

        # Connect each ball to the centroid
        for x, y, z in zip(x_coords, y_coords, z_coords):
            ax.plot([centroid[0], x], [centroid[1], y], [centroid[2], z], color='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title("Tracker Plots")
    plt.show()

# Make plot interactive
plt.ion()

# Plot each tracker
plot_trackers(grouped_trackers)
Enter shape for Tracker 1 (1 or 2): 1


SyntaxError: invalid syntax (1106747928.py, line 111)

In [16]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the known tracker shapes
tracker_shapes = {
    "1": [
        (0.00, 0.00, 0.00),
        (0.00, 28.59, 41.02),
        (0.00, 0.00, 88.00),
        (0.00, -44.32, 40.45)
    ],
    "2": [
        (0.00, 0.00, 0.00),
        (0.00, 47.38, 28.99),
        (0.00, 0.00, 89.10),
        (0.00, -35.36, 35.36)
    ]
}

def distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def relative_distances(shape):
    """Calculate relative distances between points in a shape."""
    return [distance(shape[i], shape[j]) for i in range(len(shape)) for j in range(i+1, len(shape))]

def match_shape(combination, shape_distances):
    """Determine if a combination of balls matches a given shape based on distances."""
    combo_distances = relative_distances([ball['coordinates'] for ball in combination])
    return np.allclose(combo_distances, shape_distances, atol=1e-3)  # atol is a tolerance level for matching distances

def classify_tracker_balls(tracker_balls, shape, tracker_id):
    """
    Classify each ball of the tracker based on the chosen shape.
    """
    classified_trackers = []
    shape_distances = relative_distances(shape)

    # Iterate over all combinations of 4 balls
    for combination in itertools.combinations(tracker_balls, 4):
        combo_balls = [ball['coordinates'] for ball in combination]
        if match_shape(combo_balls, shape_distances):
            classified_trackers.append({
                "balls": [{"id": f"Tracker_{tracker_id}_ball_{i+1}", "coordinates": ball['coordinates']} for i, ball in enumerate(combination)],
                "shape": shape
            })

    return classified_trackers

classified_trackers = []
for i in range(1, num_trackers + 1):
    # User input for each tracker shape
    user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
    chosen_shape = tracker_shapes[user_choice]

    # Classify the tracker balls
    classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
    classified_trackers.extend(classified_tracker)


def calculate_centroid(points):
    """Calculate the centroid of a set of points."""
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    z_coords = [p[2] for p in points]
    centroid = (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    return centroid

def plot_trackers(trackers):
    """Plot multiple trackers with their balls in 3D."""
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for tracker in trackers:
        # Extract coordinates for plotting
        x_coords = [ball[0] for ball in tracker['balls']]
        y_coords = [ball[1] for ball in tracker['balls']]
        z_coords = [ball[2] for ball in tracker['balls']]

        # Plot the balls
        ax.scatter(x_coords, y_coords, z_coords, color='b', s=50)

        # Calculate and plot centroid
        centroid = calculate_centroid(tracker['balls'])
        ax.scatter(*centroid, color='r', s=100)

        # Connect each ball to the centroid
        for x, y, z in zip(x_coords, y_coords, z_coords):
            ax.plot([centroid[0], x], [centroid[1], y], [centroid[2], z], color='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title("Tracker Plots")
    plt.show()

# Make plot interactive
plt.ion()

# Plot each tracker
plot_trackers(grouped_trackers)


Enter shape for Tracker 1 (1 or 2): 1


TypeError: tuple indices must be integers or slices, not str

In [13]:
%matplotlib notebook
import numpy as np
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the known tracker shapes
tracker_shapes = {
    "1": [
        (0.00, 0.00, 0.00),
        (0.00, 28.59, 41.02),
        (0.00, 0.00, 88.00),
        (0.00, -44.32, 40.45)
    ],
    "2": [
        (0.00, 0.00, 0.00),
        (0.00, 47.38, 28.99),
        (0.00, 0.00, 89.10),
        (0.00, -35.36, 35.36)
    ]
}

def distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def classify_tracker_balls(tracker_balls, shape, tracker_id):
    """
    Classify each ball of the tracker based on the chosen shape.
    """
    classified_trackers = []

    # Iterate over all combinations of 4 balls
    for combination in itertools.combinations(tracker_balls, 4):
        distances = [distance(combination[i], shape[j]) for i in range(4) for j in range(4)]
        if all(d < threshold for d in distances):
            classified_trackers.append({
                "balls": [{"id": f"Tracker_{tracker_id}_ball_{i+1}", "coordinates": ball} for i, ball in enumerate(combination)],
                "shape": shape
            })

    return classified_trackers

# Extract the coordinates
tracker_balls = [coordinates for _, coordinates in tracker_clusters.items()]

# Determine the number of trackers
num_trackers = len(tracker_balls) // 4

classified_trackers = []
for i in range(1, num_trackers + 1):
    # User input for each tracker shape
    user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
    chosen_shape = tracker_shapes[user_choice]

    # Classify the tracker balls
    classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
    classified_trackers.extend(classified_tracker)

# Print the details of each tracker
for tracker in classified_trackers:
    print(f"Tracker {tracker['balls'][0]['id'].split('_')[1]}:")
    for ball in tracker['balls']:
        print(f"{ball['id']}: {ball['coordinates']}")
    print(f"Shape: {tracker['shape']}")

classified_trackers = []
for i in range(1, num_trackers + 1):
    # User input for each tracker shape
    user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
    chosen_shape = tracker_shapes[user_choice]

    # Classify the tracker balls
    classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
    classified_trackers.extend(classified_tracker)

# Print the details of each tracker
for tracker in classified_trackers:
    print(f"Tracker {tracker['balls'][0]['id'].split('_')[1]}:")
    for ball in tracker['balls']:
        print(f"{ball['id']}: {ball['coordinates']}")
    print(f"Shape: {tracker['shape']}")
    
def calculate_centroid(points):
    """Calculate the centroid of a set of points."""
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    z_coords = [p[2] for p in points]
    centroid = (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    return centroid

def plot_trackers(trackers):
    """
    Plot multiple trackers with their balls in 3D.
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for tracker in trackers:
        # Extract coordinates for plotting
        x_coords = [ball[0] for ball in tracker['balls']]
        y_coords = [ball[1] for ball in tracker['balls']]
        z_coords = [ball[2] for ball in tracker['balls']]

        # Plot the balls
        ax.scatter(x_coords, y_coords, z_coords, color='b', s=50)

        # Calculate and plot centroid
        centroid = calculate_centroid(tracker['balls'])
        ax.scatter(*centroid, color='r', s=100)

        # Connect each ball to the centroid
        for x, y, z in zip(x_coords, y_coords, z_coords):
            ax.plot([centroid[0], x], [centroid[1], y], [centroid[2], z], color='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title("Tracker Plots")
    plt.show()

# Make plot interactive
plt.ion()

# Plot each tracker
plot_trackers(grouped_trackers)


Enter shape for Tracker 1 (1 or 2): 1


NameError: name 'threshold' is not defined

# Grouping The Clusters Into 2 Trackers

In [9]:
%matplotlib notebook
import numpy as np
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the known tracker shapes
tracker_shapes = {
    "1": [
        (0.00, 0.00, 0.00),
        (0.00, 28.59, 41.02),
        (0.00, 0.00, 88.00),
        (0.00, -44.32, 40.45)
    ],
    "2": [
        (0.00, 0.00, 0.00),
        (0.00, 47.38, 28.99),
        (0.00, 0.00, 89.10),
        (0.00, -35.36, 35.36)
    ]
}

def distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def classify_tracker_balls(tracker_balls, shape, tracker_id):
    """
    Classify each ball of the tracker based on the closest match to the chosen shape.
    """
    classified_trackers = []

    # Iterate over all combinations of 4 balls
    for combination in itertools.combinations(tracker_balls, 4):
        # Calculate the sum of distances from each ball to each shape point
        sum_distances = sum(min(distance(ball, shape_point) for shape_point in shape) for ball in combination)

        # Classify the combination as a tracker based on the sum of distances
        # This can be adjusted to use a different logic if needed
        classified_trackers.append({
            "balls": [{"id": f"Tracker_{tracker_id}_ball_{i+1}", "coordinates": ball} for i, ball in enumerate(combination)],
            "shape": shape,
            "sum_distances": sum_distances  # This can be used for further analysis or filtering
        })

    return classified_trackers
# Extract the coordinates
tracker_balls = [coordinates for _, coordinates in tracker_clusters.items()]

# Determine the number of trackers
num_trackers = len(tracker_balls) // 4

classified_trackers = []
for i in range(1, num_trackers + 1):
    # User input for each tracker shape
    user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
    chosen_shape = tracker_shapes[user_choice]

    # Classify the tracker balls
    classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
    classified_trackers.extend(classified_tracker)

# Print the details of each tracker
for tracker in classified_trackers:
    print(f"Tracker {tracker['balls'][0]['id'].split('_')[1]}:")
    for ball in tracker['balls']:
        print(f"{ball['id']}: {ball['coordinates']}")
    print(f"Shape: {tracker['shape']}")
    
def calculate_centroid(points):
    """Calculate the centroid of a set of points."""
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    z_coords = [p[2] for p in points]
    centroid = (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    return centroid

def plot_trackers(trackers):
    """
    Plot multiple trackers with their balls in 3D.
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for tracker in trackers:
        # Extract coordinates for plotting
        x_coords = [ball[0] for ball in tracker['balls']]
        y_coords = [ball[1] for ball in tracker['balls']]
        z_coords = [ball[2] for ball in tracker['balls']]

        # Plot the balls
        ax.scatter(x_coords, y_coords, z_coords, color='b', s=50)

        # Calculate and plot centroid
        centroid = calculate_centroid(tracker['balls'])
        ax.scatter(*centroid, color='r', s=100)

        # Connect each ball to the centroid
        for x, y, z in zip(x_coords, y_coords, z_coords):
            ax.plot([centroid[0], x], [centroid[1], y], [centroid[2], z], color='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title("Tracker Plots")
    plt.show()

# Make plot interactive
plt.ion()

# Plot each tracker
plot_trackers(grouped_trackers)

Enter shape for Tracker 1 (1 or 2): 1
Enter shape for Tracker 2 (1 or 2): 2
Tracker 1:
Tracker_1_ball_1: (286.0, 301.0, 3.5)
Tracker_1_ball_2: (268.0, 260.0, 105.5)
Tracker_1_ball_3: (374.0, 375.0, 22.5)
Tracker_1_ball_4: (203.0, 396.5, 37.5)
Shape: [(0.0, 0.0, 0.0), (0.0, 28.59, 41.02), (0.0, 0.0, 88.0), (0.0, -44.32, 40.45)]
Tracker 1:
Tracker_1_ball_1: (286.0, 301.0, 3.5)
Tracker_1_ball_2: (268.0, 260.0, 105.5)
Tracker_1_ball_3: (374.0, 375.0, 22.5)
Tracker_1_ball_4: (332.0, 479.0, 55.5)
Shape: [(0.0, 0.0, 0.0), (0.0, 28.59, 41.02), (0.0, 0.0, 88.0), (0.0, -44.32, 40.45)]
Tracker 1:
Tracker_1_ball_1: (286.0, 301.0, 3.5)
Tracker_1_ball_2: (268.0, 260.0, 105.5)
Tracker_1_ball_3: (374.0, 375.0, 22.5)
Tracker_1_ball_4: (375.5, 339.0, 105.5)
Shape: [(0.0, 0.0, 0.0), (0.0, 28.59, 41.02), (0.0, 0.0, 88.0), (0.0, -44.32, 40.45)]
Tracker 1:
Tracker_1_ball_1: (286.0, 301.0, 3.5)
Tracker_1_ball_2: (268.0, 260.0, 105.5)
Tracker_1_ball_3: (374.0, 375.0, 22.5)
Tracker_1_ball_4: (198.0, 308.0, 147

NameError: name 'grouped_trackers' is not defined

In [ ]:
%matplotlib notebook
import numpy as np
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the known tracker shapes
tracker_shapes = {
    "1": [
        (0.00, 0.00, 0.00),
        (0.00, 28.59, 41.02),
        (0.00, 0.00, 88.00),
        (0.00, -44.32, 40.45)
    ],
    "2": [
        (0.00, 0.00, 0.00),
        (0.00, 47.38, 28.99),
        (0.00, 0.00, 89.10),
        (0.00, -35.36, 35.36)
    ]
}

def distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def classify_tracker_balls(tracker_balls, shape, tracker_id):
    """
    Classify each ball of the tracker based on the chosen shape.
    """
    classified_trackers = []

    # Iterate over all combinations of 4 balls
    for combination in itertools.combinations(tracker_balls, 4):
        distances = [distance(combination[i], shape[j]) for i in range(4) for j in range(4)]
        if all(d < threshold for d in distances):
            classified_trackers.append({
                "balls": [{"id": f"Tracker_{tracker_id}_ball_{i+1}", "coordinates": ball} for i, ball in enumerate(combination)],
                "shape": shape
            })

    return classified_trackers

# Load and process tracker data (similar to the first script)
def load_and_process_tracker_data():
    # Assuming tracker_balls is already defined and filled with the tracker data
    tracker_balls = [coordinates for _, coordinates in tracker_clusters.items()]
    classified_trackers = []
    for i in range(1, num_trackers + 1):
        user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
        chosen_shape = tracker_shapes[user_choice]
        classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
        classified_trackers.extend(classified_tracker)
    return classified_trackers

# Modify the update_projection function to include tracker data
def update_projection_with_trackers():
    ax.cla()
    current_proj = get_current_projection()
    ax.imshow(current_proj, cmap='gray', origin='lower', aspect='auto')

    # Overlay the tracker data on the current projection
    for tracker in classified_trackers:
        for ball in tracker['balls']:
            x, y, z = ball['coordinates']
            # Plot the tracker based on the current projection
            if current_plane.get() == 'XY' and z_slice == z:
                ax.plot(x, y, 'ro')
            elif current_plane.get() == 'XZ' and y_slice == y:
                ax.plot(x, z, 'ro')
            elif current_plane.get() == 'YZ' and x_slice == x:
                ax.plot(y, z, 'ro')

    plt.title('CT Slice with Tracker Overlay')
    plt.draw()

# Load tracker data
classified_trackers = load_and_process_tracker_data()



# Path to your DICOM files
dicom_folder = 'AXIAL'  # Ensure this is the correct path to the DICOM files

# Read all DICOM files in the folder and filter for .dcm files
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder)]
if not dicom_files:
    raise ValueError(f"No DICOM files found in the folder {dicom_folder}")

dicom_files = sort_dicom_files(dicom_files)
if not dicom_files:
    raise ValueError("No DICOM files with 'ImagePositionPatient' attribute found after sorting.")

# Read the first DICOM file to get metadata
first_dicom = pydicom.dcmread(dicom_files[0])

# Initialize the 3D array to store CT data
CTraw = np.zeros((len(dicom_files), first_dicom.Rows, first_dicom.Columns), dtype=np.int16)

# Read DICOM files and fill the 3D array
for i, dicom_file in enumerate(dicom_files):
    CTraw[i, :, :] = pydicom.dcmread(dicom_file).pixel_array

# Apply threshold mask
mask_value = 1200
CTraw[CTraw < mask_value] = 0

# Change data ordering to match Cartesian coordinates
CT = np.flip(CTraw, axis=0)

# Use XY projection
XZproj = np.sum(CT, axis=0)
YZproj = np.sum(CT, axis=1)
XYproj = np.sum(CT, axis=2)

# Create the main window
fig, ax = plt.subplots()

# Initialize the plot with default values
ax.imshow(XYproj, cmap='gray', origin='lower', aspect='auto')
plt.title('Click on endpoints of lines you would like to measure (two points per line)')

# Connect the click event to the on_click function
fig.canvas.mpl_connect('button_press_event', on_click)

# Create Tkinter window for plane selection
plane_window = Tk()
plane_window.title("Plane Selection")
plane_options = ['XY', 'XZ', 'YZ']
current_plane = StringVar(plane_window)
current_plane.set('XY')  # Default plane
plane_menu = OptionMenu(plane_window, current_plane, *plane_options)
plane_menu.pack()

# Attach the callback to the plane variable
current_plane.trace('w', on_plane_change)

# Close the plane selection window when the main window is closed
plane_window.protocol("WM_DELETE_WINDOW", plane_window.destroy)

# Function to run Tkinter main loop in a separate thread
def tkinter_mainloop():
    plane_window.mainloop()

# Start the Tkinter main loop in a separate thread
tkinter_thread = threading.Thread(target=tkinter_mainloop)
tkinter_thread.start()

# Display the Matplotlib window
plt.show()

# Initialize the plot with default values and tracker overlay
update_projection_with_trackers()

# Extract the coordinates
tracker_balls = [coordinates for _, coordinates in tracker_clusters.items()]

# Determine the number of trackers
num_trackers = len(tracker_balls) // 4

classified_trackers = []
for i in range(1, num_trackers + 1):
    # User input for each tracker shape
    user_choice = input(f"Enter shape for Tracker {i} (1 or 2): ")
    chosen_shape = tracker_shapes[user_choice]

    # Classify the tracker balls
    classified_tracker = classify_tracker_balls(tracker_balls, chosen_shape, i)
    classified_trackers.extend(classified_tracker)

# Print the details of each tracker
for tracker in classified_trackers:
    print(f"Tracker {tracker['balls'][0]['id'].split('_')[1]}:")
    for ball in tracker['balls']:
        print(f"{ball['id']}: {ball['coordinates']}")
    print(f"Shape: {tracker['shape']}")
    
def calculate_centroid(points):
    """Calculate the centroid of a set of points."""
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    z_coords = [p[2] for p in points]
    centroid = (np.mean(x_coords), np.mean(y_coords), np.mean(z_coords))
    return centroid

def plot_trackers(trackers):
    """
    Plot multiple trackers with their balls in 3D.
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for tracker in trackers:
        # Extract coordinates for plotting
        x_coords = [ball[0] for ball in tracker['balls']]
        y_coords = [ball[1] for ball in tracker['balls']]
        z_coords = [ball[2] for ball in tracker['balls']]

        # Plot the balls
        ax.scatter(x_coords, y_coords, z_coords, color='b', s=50)

        # Calculate and plot centroid
        centroid = calculate_centroid(tracker['balls'])
        ax.scatter(*centroid, color='r', s=100)

        # Connect each ball to the centroid
        for x, y, z in zip(x_coords, y_coords, z_coords):
            ax.plot([centroid[0], x], [centroid[1], y], [centroid[2], z], color='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title("Tracker Plots")
    plt.show()

# Make plot interactive
plt.ion()





# Plot each tracker
plot_trackers(grouped_trackers)

# Bone Classification Algorithim (BCA)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from IPython.display import Image, display
from tkinter import Tk, simpledialog, StringVar, OptionMenu
from skimage.transform import resize
import threading

# Switch to Tkinter backend for interactive features
plt.switch_backend("TkAgg")
# Define global variables
current_plane = None
XYproj = None
XZproj = None
YZproj = None
selected_points = []
line_counter = [0]
lines_folder = 'LINES'

def calculate_angle(line1, line2):
    # Convert lines to numpy arrays
    line1 = np.array(line1)
    line2 = np.array(line2)

    # Calculate vectors
    vector1 = line1[1] - line1[0]
    vector2 = line2[1] - line2[0]

    # Calculate dot product
    dot_product = np.dot(vector1, vector2)

    # Calculate magnitudes
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    # Calculate cosine of the angle
    cos_theta = dot_product / (magnitude1 * magnitude2)

    # Calculate angle in radians
    angle_radians = np.arccos(cos_theta)

    # Convert angle to degrees
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees

# Function to sort DICOM files
def sort_dicom_files(dicom_files):
    # Create a list of tuples containing the file path and slice location
    slices = []
    for file in dicom_files:
        try:
            dcm = pydicom.dcmread(file, force=True)
            # Check if 'ImagePositionPatient' attribute is present
            if hasattr(dcm, 'ImagePositionPatient'):
                slices.append((file, dcm))
            else:
                print(f"Skipping {file}: 'ImagePositionPatient' attribute not found")
        except Exception as e:
            print(f"Could not read {file}: {e}")
    # Sort based on ImagePositionPatient[2] if available
    slices.sort(key=lambda x: float(x[1].ImagePositionPatient[2]))
    # Return the sorted file paths
    return [x[0] for x in slices]

def save_line(line, line_id, plane):
    line_filename = os.path.join(lines_folder, f'line_{line_id}.txt')
    np.savetxt(line_filename, line)

    fig, ax = plt.subplots()

    if plane == 'XY':
        ax.imshow(XYproj, cmap='gray', origin='lower', aspect='auto')
    elif plane == 'XZ':
        ax.imshow(XZproj, cmap='gray', origin='lower', aspect='auto')
    elif plane == 'YZ':
        ax.imshow(YZproj, cmap='gray', origin='lower', aspect='auto')

    line = np.array(line)
    ax.plot(line[:, 0], line[:, 1], 'r-', lw=2)
    ax.set_title(f'Line Detected ({plane} Plane)')

    fig.savefig(os.path.join(lines_folder, f'line_{line_id}.png'))
    plt.close(fig)

def calculate_and_print_angles():
    global lines_folder
    print(f"Checking files in {lines_folder}")
    for filename in os.listdir(lines_folder):
        print(filename)

    line1_path = os.path.join(lines_folder, 'line_0.txt')
    line2_path = os.path.join(lines_folder, 'line_1.txt')

    if os.path.exists(line1_path) and os.path.exists(line2_path):
        line1 = np.loadtxt(line1_path)
        line2 = np.loadtxt(line2_path)

        angle_degrees = calculate_angle(line1, line2)
        print(f"Angle between the two lines: {angle_degrees:.2f} degrees")
    else:
        print("Files not found. Make sure lines are saved correctly.")

def on_click(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        selected_points.append((x, y))
        ax.plot(x, y, 'ro', markersize=5)
        plt.draw()

        # If two points are selected, show popup dialog
        if len(selected_points) == 2:
            root = Tk()
            root.withdraw()
            # Ask the user for the next action
            user_response = simpledialog.askstring("Line Selection", "Place next line or redo points? (next/redo)")
            root.destroy()

            if user_response.lower() == 'next':
                plane = current_plane.get()
                save_line(selected_points, line_counter[0], plane)
                selected_points.clear()
                line_counter[0] += 1

                # Check if two lines have been made
                if line_counter[0] == 2:
                    print("Two lines have been made.")
                    plt.close(fig)
                    # Calculate and save each line
                    for i in range(0, len(selected_points), 2):
                        if i + 1 < len(selected_points):
                            save_line(selected_points[i:i+2], i // 2, plane)
                    # Call the function to calculate and print angles
                    calculate_and_print_angles()
            elif user_response.lower() == 'redo':
                selected_points.pop()
                selected_points.pop()
                ax.cla()
                update_projection()
                plt.title('Click on endpoints of lines you would like to measure (two points per line)')

def get_current_projection():
    if current_plane.get() == 'XY':
        return XYproj
    elif current_plane.get() == 'XZ':
        return XZproj
    elif current_plane.get() == 'YZ':
        return YZproj

def update_projection():
    ax.cla()
    ax.imshow(get_current_projection(), cmap='gray', origin='lower', aspect='auto')
    plt.title('Click on endpoints of lines you would like to measure (two points per line)')
    plt.draw()
   
# Callback function to update the projection when the plane is changed
def on_plane_change(*args):
    update_projection()


# Path to your DICOM files
dicom_folder = 'AXIAL'  # Ensure this is the correct path to the DICOM files

# Read all DICOM files in the folder and filter for .dcm files
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder)]
if not dicom_files:
    raise ValueError(f"No DICOM files found in the folder {dicom_folder}")

dicom_files = sort_dicom_files(dicom_files)
if not dicom_files:
    raise ValueError("No DICOM files with 'ImagePositionPatient' attribute found after sorting.")

# Read the first DICOM file to get metadata
first_dicom = pydicom.dcmread(dicom_files[0])

# Initialize the 3D array to store CT data
CTraw = np.zeros((len(dicom_files), first_dicom.Rows, first_dicom.Columns), dtype=np.int16)

# Read DICOM files and fill the 3D array
for i, dicom_file in enumerate(dicom_files):
    CTraw[i, :, :] = pydicom.dcmread(dicom_file).pixel_array

# Apply threshold mask
mask_value = 1200
CTraw[CTraw < mask_value] = 0

# Change data ordering to match Cartesian coordinates
CT = np.flip(CTraw, axis=0)

# Use XY projection
XZproj = np.sum(CT, axis=0)
YZproj = np.sum(CT, axis=1)
XYproj = np.sum(CT, axis=2)

# Create and ensure LINES folder exists
lines_folder = 'LINES'
if not os.path.exists(lines_folder):
    os.makedirs(lines_folder)

# Define lists to store selected points
selected_points = []
line_counter = [0]



# Create the main window
fig, ax = plt.subplots()

# Initialize the plot with default values
ax.imshow(XYproj, cmap='gray', origin='lower', aspect='auto')
plt.title('Click on endpoints of lines you would like to measure (two points per line)')

# Connect the click event to the on_click function
fig.canvas.mpl_connect('button_press_event', on_click)

# Create Tkinter window for plane selection
plane_window = Tk()
plane_window.title("Plane Selection")
plane_options = ['XY', 'XZ', 'YZ']
current_plane = StringVar(plane_window)
current_plane.set('XY')  # Default plane
plane_menu = OptionMenu(plane_window, current_plane, *plane_options)
plane_menu.pack()

# Attach the callback to the plane variable
current_plane.trace('w', on_plane_change)

# Close the plane selection window when the main window is closed
plane_window.protocol("WM_DELETE_WINDOW", plane_window.destroy)

# Function to run Tkinter main loop in a separate thread
def tkinter_mainloop():
    plane_window.mainloop()

# Start the Tkinter main loop in a separate thread
tkinter_thread = threading.Thread(target=tkinter_mainloop)
tkinter_thread.start()

# Display the Matplotlib window
plt.show()

Exception in thread Thread-5 (tkinter_mainloop):
Traceback (most recent call last):
  File "C:\Users\seife\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\seife\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\seife\AppData\Local\Temp\ipykernel_37696\423527721.py", line 234, in tkinter_mainloop
  File "C:\Users\seife\anaconda3\Lib\tkinter\__init__.py", line 1485, in mainloop
    self.tk.mainloop(n)
RuntimeError: Calling Tcl from different apartment


Two lines have been made.
Checking files in LINES
line_0.png
line_0.txt
line_0_XY.png
line_1.png
line_1.txt
line_1_XY.png
Angle between the two lines: 15.53 degrees
